In [129]:
import numpy as np
from anytree import RenderTree
import itertools
import logging

import log_set

In [130]:
%load_ext autoreload
%autoreload 2

from tictac_rl import TicTacToe, MinMaxTree

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [122]:
logger = logging.getLogger("tictoc")

In [131]:
env = TicTacToe(3, 3, 3)

In [5]:
tree = MinMaxTree.load_from_dump("trees/3/cross/")

In [135]:
for start_x, start_y in itertools.product(range(env.n_rows), range(env.n_cols)):
    logger.info("start")
    env.reset()
    is_max = True
    hashes = []
    (hash_state, *_), *_  = env.step((start_x, start_y))
    hashes.append(hash_state)
    while True:
        is_max = not is_max
        move = tree.best_move(hashes, is_max)
        (hash_state, *_), reward, is_end  = env.step(move)
        hashes.append(hash_state)
        if is_end:
            if reward != 0:
                for hash_str in hashes:
                    logger.info(hash_str)
                    logger.info(env.from_hash(hash_str))
                logger.handlers[0].flush()
                raise ValueError()
            break